In [ ]:
import cv2
import numpy as np
import os, glob
import random
from random import randint
input_folder = "input_images" # folder of classes folders with images
output_folder = "generated_images" # results will be saven in this folder
classes = 10 #number of classes 
augmentation_number = 10 # how many images will be created from each input image
destination_size = (128,1024) # output image size
word_height = int(destination_size[0]/2) # height of word in output image

In [ ]:
# ADDING NOISES (SEVERAL VARIANTS)

#variant 1
def add_gaussian_noise(image_in, noise_sigma = 15):
    noise_sigma = randint(1,noise_sigma)
    temp_image = np.float64(np.copy(image_in))

    h = temp_image.shape[0]
    w = temp_image.shape[1]
    noise = np.random.randn(h, w) * noise_sigma

    noisy_image = np.zeros(temp_image.shape, np.float64)
    if len(temp_image.shape) == 2:
        noisy_image = temp_image + noise
    else:
        noisy_image[:,:,0] = temp_image[:,:,0] + noise
        noisy_image[:,:,1] = temp_image[:,:,1] + noise
        noisy_image[:,:,2] = temp_image[:,:,2] + noise

    return noisy_image



# variant 2
def sp_noise(image,prob=0.05):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output

# variant 3
def numpy_noise(image, mean=0.0, std = 1.0):
    # mean = 0.5  # some constant
    # std = 1.0    # some constant (standard deviation)
    noisy_img = image + np.random.normal(mean, std, image.shape)
    return noise_img

# КОД ДЛЯ ТЕСТА
# image = cv2.imread('noises/image.jpg',0) # Only for grayscale image
# noisy_img = add_gaussian_noise(image)
# cv2.imwrite('noises/gaussian_image.jpg', noisy_img)
# noisy_img = sp_noise(image)
# cv2.imwrite('noises/sp_image.jpg', noisy_img)
# noisy_img = numpy_noise(image)
# cv2.imwrite('noises/numpy_image.jpg', noisy_img)

In [ ]:
# detect borders of the word in input image, and crop it
def detect_word(img):
    if len(img.shape)>2 and img.shape[2]>1:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (destination_size[1], destination_size[0])) 
    #th = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    th = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    hist = np.average(th,axis=0)
    rb = img.shape[1]
    for i in range(len(hist)-1, 3, -1):
        h=hist[i]
        if hist[i]>252 or hist[i-1]>252 or hist[i-2]>252 :
            rb = i
        else:
            break
    crop_img = th[:,:rb]
    new_width = int(crop_img.shape[1]*(word_height/crop_img.shape[0]))
    detected_word = cv2.resize(crop_img, (new_width,word_height), interpolation = cv2.INTER_AREA)
    return detected_word

# put cropped image into template image with random position
def random_position(small_image, big_image):
    H,W = big_image.shape
    h,w = small_image.shape
    x_range = W-w
    y_range = H-h
    x_pos = randint(0,x_range)
    y_pos = randint(0,y_range)
    big_image[y_pos:y_pos+h,x_pos:x_pos+w] = small_image
    return big_image

# main function with generate image from single image
def preproc_img(img):
    crop_img = detect_word(img)   
    new_img = np.ones((destination_size[0],destination_size[1]),dtype=np.uint8)*255
    new_img = random_position(crop_img,new_img)
    new_img = add_gaussian_noise(new_img,25)
    return new_img



In [ ]:
# чтение оригинального файла, аугментация и сохранение
dataset = input_folder
for c in range(0,classes):
    class_dir = os.path.join(dataset,str(c))
    all_files = glob.glob(class_dir+"/*.jpg")
    
    for j, full_path in enumerate(all_files):
        file = os.path.basename(full_path)
        img = cv2.imread(full_path)
        for i in range(augmentation_number):
            new_img = preproc_img(img)
            # cv2.imshow("test",new_img)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # break
            directory = output_folder+"/"+str(c)+"/"
            if not os.path.exists(directory):
                os.makedirs(directory)
            cv2.imwrite(directory+file[:-4]+str(i)+file[-4:],new_img)

In [ ]:
output_folder

In [ ]:
# чтение из готового датасета и разделение на train и test
train_dir = "train"
test_dir = "test"
test_portion = 0.15
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    
dataset = output_folder
for c in range(0,classes):
    class_dir = os.path.join(dataset,str(c))
    all_files = glob.glob(class_dir+"/*.jpg")
    random.shuffle(all_files)
    for j, full_path in enumerate(all_files):
        file = os.path.basename(full_path)
        img = cv2.imread(full_path)
        img = cv2.resize(img,(int(img.shape[1]/2),int(img.shape[0]/2)))
        if j<test_portion*len(all_files):
            directory = test_dir+"/"+str(c)+"/"
        else:
            directory = train_dir+"/"+str(c)+"/"
        if not os.path.exists(directory):
            os.makedirs(directory)
        cv2.imwrite(directory+file,img)

In [1]:
import augment_imgaug
import cv2

In [2]:
full_path = "input_images/8/823821.jpg"
aug_number = 5 
OUTPUT_IMG_DIR = "generated_images"
img = cv2.imread(full_path,cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, None, fx=0.5, fy=0.5,interpolation=cv2.INTER_CUBIC)
new_img = augment_imgaug.preproc_img(img)
cv2.imshow("input",img)
cv2.imshow("output",new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

batch = augment_imgaug.compile_batch(img, aug_number)
batch_aug = augment_imgaug.augment(batch)
# save 
directory = OUTPUT_IMG_DIR+"/"
augment_imgaug.save_batch(directory, batch_aug, aug_number)